# Hand Mirroring
This notebook demonstrates how you can get the robot arm to mirror your hand using a webcam. You will need a laptop, a working MQTT broker, a running UR driver, a running MQTT forwarder, and an instance of the MyUR3e and Sub_Node class.

## On your Laptop:
Install opencv and paho-mqtt using pip. Then run this code locally on your laptop and find a blue object to hold in your hand for the camera to track. You can also change the color thresholds to track a different color!

In [ ]:
import cv2
import numpy as np
import paho.mqtt.client as mqtt
import json

# MQTT setup
broker_address = "130.64.16.222"  # Replace with your MQTT broker address
broker_port = 1884 # Replace with your MQTT broker port
client = mqtt.Client("opencv_tracker") # Name of client
client.connect(broker_address,broker_port)

# Start capturing video from the webcam
cap = cv2.VideoCapture(1)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the range of color to track
# In this instance we track a range of blue in HSV (Hue, Saturation, Value)
lower_blue = np.array([90, 50, 70])
upper_blue = np.array([130, 255, 255])

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break

    # Mirror the frame horizontally
    frame = cv2.flip(frame, 1)
    
    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask for the blue color
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # If contours are found, track the largest one
    if contours:
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Draw the bounding box around the blue object
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Calculate the center of the bounding box from 0-1
        center_x = (x + w // 2) / frame_width
        center_y = (y + h // 2) / frame_height

        # Display the coordinates
        cv2.putText(frame, f"Coordinates: ({center_x}, {center_y})", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Send coordinates over MQTT to the "camera" topic
        payload = json.dumps({"x": center_x, "y": center_y})
        client.publish("camera", payload)
        print("Published: ", payload)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the windows
cap.release()
cv2.destroyAllWindows()

# Disconnect MQTT client
client.disconnect()


## On your Jupyter Hub:
Make sure your UR Driver is running and that you have started an MQTT forwarder for the "camera" topic. Then run the code below to start controlling the UR arm based on the positions of the color tracker.

In [ ]:
from myur import MyUR3e
from myur import sub_node

robot = MyUR3e()
tracker = sub_node.Sub_Node('mqtt/camera')

def in_invisible_fence(pos):
    bottom = 0.2
    top = 0.4
    if pos < bottom or pos > top:
        return False
    return True

import json
from IPython.display import clear_output

while True:
    clear_output(wait=True)
    coords = json.loads(tracker.get_data())
    y = 0.2 + (1-coords.get('y')) * 0.2 # y camera axis -> z robot axis
    x = 0.2 + (1-coords.get('x')) * 0.2 # x camera axis -> y robot axis
    print("Z: ",y)
    if in_invisible_fence(y) and in_invisible_fence(x):
        print("Moving arm")
        robot.move_global([0.1,x,y,180,0,0],time=('cv',0.1,0.1))
    else:
        print("Out of bounds")